In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib
import requests

In [2]:
# call api
url = "https://prices.runescape.wiki/api/v1/osrs/timeseries?timestep=1h&id=2"
headers = {
    "User-Agent": "RuneScape-Market-Analyzer-Bot/1.0 (Contact: taniguchi.k@ufl.edu)"
}
response = requests.get(url, headers=headers)
data = response.json()

# convert to df
df = pd.DataFrame(data["data"])
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
df["avgPrice"] = (df["avgHighPrice"] + df["avgLowPrice"]) / 2
df.head()

,timestamp,avgHighPrice,avgLowPrice,highPriceVolume,lowPriceVolume,avgPrice
0,2025-02-26 17:00:00,199,197,633719,429458,198.0
1,2025-02-26 18:00:00,200,197,473077,343642,198.5
2,2025-02-26 19:00:00,202,198,839732,312822,200.0
3,2025-02-26 20:00:00,204,199,1283529,422539,201.5
4,2025-02-26 21:00:00,202,197,1582498,452177,199.5


In [3]:
# create features
df.dropna(inplace=True)
df["prev_price"] = df["avgPrice"].shift(1)
df.dropna(inplace=True) # drop first entry
X = df[["prev_price"]]  # features
y = df["avgPrice"]  # target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 0.9441679762927407


In [5]:
# save model
joblib.dump(model, "runescape_model.pkl")

['runescape_model.pkl']

In [6]:
# load model
model = joblib.load("runescape_model.pkl")

In [7]:
# test prediction
latest_price = df["prev_price"].iloc[-1]

next_input = pd.DataFrame([[latest_price]], columns=["prev_price"])

predicted_price = model.predict(next_input)

print(f"Predicted Next Price: {predicted_price[0]}")

Predicted Next Price: 206.7192619047619
